<a href="https://colab.research.google.com/github/Akshaj0712/Speed-detection/blob/main/Style%20Transfer%20Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#if not using colab, install pytorch
#pip3 install torch==1.10.0+cu102 torchvision==0.11.1+
#cu102 torchaudio===0.10.0+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html
import torch
import torch.nn as nn
import torch.optim as optimization

import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.utils import save_image

from tqdm.notebook import tqdm
from PIL import Image


In [8]:
def load_img(path):
  img = Image.open(path)
  img = loader(img).unsqueeze(0)
  return img.to(device)

In [26]:
def save(target, i):
  #the image needs to be denormalized first
  denormalization = transforms.Normalize((-2.12, -2.04, -1.80), (4.37, 4.46, 4.44))
  #remove the additional dimension
  img = target.clone().squeeze()
  img = denormalization(img).clamp(0, 1)
  save_image(img, f'result_{i}.png')


In [23]:
#if gpu available use it and load images in higher resolution
#for cpu:
#device = 'cpu'
#img_size = 128
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
img_size = 512 if torch.cuda.is_available() else 128
#preprocessing of the images
loader = transforms.Compose([
                             transforms.Resize(img_size),
                             transforms.ToTensor(),
                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

#initial model
model = models.vgg19(pretrained=True).features

generate_img = torch.randn((1,3,512,512), requires_grad = True)
#the optimizer; in the paper they are using LBFGS, but we can go ahead and use Adam since it's generally more adequate
optimizer = optimization.Adam([generate_img], lr=0.001)

steps = 1000
alpha = 1 #content wight
beta = 10000 #style weight

#the 3 needed images
#define the load_img function first
content_img = load_img('/content.jpg')
style_img = load_img('/style.jpg')
#we can start from a random noise generated image or
#just copy the content image as a starting point
target_img = content_img.clone().requires_grad_(True)

In [28]:
def get_content_loss(target, content):
  return torch.mean((target-content)**2)

def gram_matrix(input, c, h, w):
  #c-channels; h-height; w-width
  input = input.view(c, h*w)
  #matrix multiplication on its own transposed form
  G = torch.mm(input, input.t())
  return G

def get_style_loss(target, style):
  _, c, h, w = target.size()
  G = gram_matrix(target, c, h, w) #gram matrix for the target image
  S = gram_matrix(style, c, h, w) #gram matrix for the style image
  return torch.mean((G-S)**2)/(c*h*w)

#class for loading the vgg19 model with wanted layers
class VGG(nn.Module):
  def __init__(self):
    super(VGG, self).__init__()
    self.select_features = ['0', '5', '10', '19', '28'] #conv layers
    self.vgg = models.vgg19(pretrained=True).features

  def forward(self, output):
    features = []
    for name, layer in self.vgg._modules.items():
      output = layer(output)
      if name in self.select_features:
        features.append(output)
    return features

#load the model
vgg = VGG().to(device).eval()

In [1]:
for step in tqdm(range(steps)):
  #get feature vectors representations for every image
  target_feature = vgg(target_img)
  content_feature = vgg(content_img)
  style_feature = vgg(style_img)

  #initiate the losses
  style_loss = 0
  content_loss = 0

  for target, content, style in zip(target_feature, content_feature, style_feature):
    # print(target.size()[0:2], content.size()[0:2], style.size()[0:2])
    content_loss += get_content_loss(target, content)
    style_loss += get_style_loss(style, content)
  #calculate the total loss according to the original paper
  total_loss = alpha*content_loss+beta*style_loss

  #set parameters to zero
  optimizer.zero_grad()
  #compute the gradient
  total_loss.backward()
  #update parameters
  optimizer.step()

  #uncomment if you want to follow the progress
  #if step%50 == 0:
    #print(f'step: {step}, content loss: {content_loss.item()}, style loss: {style_loss.item()}')
  #if step%100 == 0:
    #save(target_img, step)

NameError: name 'tqdm' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')